In [1]:
import glob

import pandas as pd
import screed

# Change to Quest for Orthologs 2019 data directory

In [ ]:
cd ~/data_sm/kmer-hashing/quest-for-orthologs/data/2019/

In [14]:
ls -lha

total 2.6G
drwxr-xr-x 5 olga root 4.0K Jan 10 08:02 ./
drwxr-xr-x 3 olga root 4.0K Dec 25 17:48 ../
drwxr-xr-x 5 olga czb  4.0K Dec 26 19:44 Archaea/
drwxr-xr-x 5 olga czb   16K Dec 26 19:44 Bacteria/
drwxr-xr-x 8 olga czb   32K Jan  8 08:13 Eukaryota/
-rw-r--r-- 1 olga czb  754K Jan 10 07:50 human_transcription_factors_with_uniprot_ids.csv
-rw-r--r-- 1 olga czb   68K Jan 10 07:50 human_transcription_factors_with_uniprot_ids.csv.gz
-rw-r--r-- 1 olga czb  133K Jan 10 07:50 human_transcription_factors_with_uniprot_ids.parquet
-rw-r--r-- 1 olga czb   64M Jan 10 07:43 opisthokont_not_human_transcription_factors_ensembl_compara.csv
-rw-r--r-- 1 olga czb  1.8M Jan 10 08:02 opisthokont_not_human_transcription_factors_ensembl_compara_merged_uniprot.csv.gz
-rw-r--r-- 1 olga czb  2.2M Jan 10 08:02 opisthokont_not_human_transcription_factors_ensembl_compara_merged_uniprot.parquet
-rw-r--r-- 1 olga czb   12M Jan 10 07:43 opisthokont_not_human_transcription_factors_ensembl_compara.parquet
-rw-r--r-

# Get Retinal gene names from 

In [15]:
s = 'RHO	GNAT1	GNB1	GNGT1	OPN1SW	OPN1MW	GNAT2	GNB3	GNGT2 PDE6A	PDE6B	PDE6G PDE6C	PDE6H SAG	ARR3	RGS9 CNGA1	CNGA3	CNGB1	CNGB3 GRK1	GRK7	RCVRN	GUCA1A		GUCA1B	GUCY2D	GUCY2F'
genes = s.split()
genes

['RHO',
 'GNAT1',
 'GNB1',
 'GNGT1',
 'OPN1SW',
 'OPN1MW',
 'GNAT2',
 'GNB3',
 'GNGT2',
 'PDE6A',
 'PDE6B',
 'PDE6G',
 'PDE6C',
 'PDE6H',
 'SAG',
 'ARR3',
 'RGS9',
 'CNGA1',
 'CNGA3',
 'CNGB1',
 'CNGB3',
 'GRK1',
 'GRK7',
 'RCVRN',
 'GUCA1A',
 'GUCA1B',
 'GUCY2D',
 'GUCY2F']

In [3]:
len(genes)

28

In [6]:
ensembl_rest.lookup_post(genes[0])

'Content-Type application/json had a problem with your request.\r\n***ERROR***\r\nJSON text must be an object or array (but found number, string, true, false or null, use allow_nonref to allow this) at /nfs/public/release/ensweb/live/rest/www_98/.plenv/versions/5.26.1/lib/perl5/site_perl/5.26.1/Catalyst/Action/Deserialize/JSON.pm line 35, <$fh> line 1.\n'

In [9]:
s = 'P08100	P11488	P62873	P63211	P03999	P04001	P19087	P16520	O14610 PDE6A	PDE6B	PDE6G PDE6C	PDE6H P10523	P36575	O75916			P29973	Q16281	Q14028	B9EK43 Q15835	Q8WTQ7	P35243	P43080		Q9UMX6	Q02846	P51841'
uniprot_ids = [x for x in s.split() if x]
uniprot_ids

['P08100',
 'P11488',
 'P62873',
 'P63211',
 'P03999',
 'P04001',
 'P19087',
 'P16520',
 'O14610',
 'PDE6A',
 'PDE6B',
 'PDE6G',
 'PDE6C',
 'PDE6H',
 'P10523',
 'P36575',
 'O75916',
 'P29973',
 'Q16281',
 'Q14028',
 'B9EK43',
 'Q15835',
 'Q8WTQ7',
 'P35243',
 'P43080',
 'Q9UMX6',
 'Q02846',
 'P51841']

In [16]:
genes_to_ids = pd.DataFrame({'symbol': genes, 'uniprot_id': uniprot_ids})
print(genes_to_ids.shape)
genes_to_ids.head()

(28, 2)


,symbol,uniprot_id
0,RHO,P08100
1,GNAT1,P11488
2,GNB1,P62873
3,GNGT1,P63211
4,OPN1SW,P03999


In [17]:
genes_to_ids.tail()

,symbol,uniprot_id
23,RCVRN,P35243
24,GUCA1A,P43080
25,GUCA1B,Q9UMX6
26,GUCY2D,Q02846
27,GUCY2F,P51841


In [10]:
len(uniprot_ids)

28

# Read Human ID mapping

In [21]:
human_id_mapping = pd.read_csv('Eukaryota/UP000005640_9606.idmapping', sep='\t', header=None, names=['uniprot_id', 'id_type', 'db_id'])
human_id_mapping.columns = 'source__' + human_id_mapping.columns
print(human_id_mapping.shape)
human_id_mapping.head()

(2668934, 3)


,source__uniprot_id,source__id_type,source__db_id
0,P62258-2,UniParc,UPI00001E6021
1,P62258-2,Ensembl_TRS,ENST00000571732
2,P62258-2,Ensembl_PRO,ENSP00000461762
3,P62258-2,Ensembl_TRS,ENST00000616643
4,P62258-2,Ensembl_PRO,ENSP00000481059


## Extract with gene symbsls

In [23]:
human_id_mapping_symbols = human_id_mapping.query('source__db_id in @genes_to_ids.symbol')
print(human_id_mapping_symbols.shape)
human_id_mapping_symbols.head()

(194, 3)


,source__uniprot_id,source__id_type,source__db_id
73832,P29973,Gene_Name,CNGA1
73873,P29973,BioMuta,CNGA1
73884,P29973,GeneCards,CNGA1
73885,P29973,GeneReviews,CNGA1
73904,P29973,ChiTaRS,CNGA1


In [25]:
human_id_mapping_symbols.source__db_id.nunique()

28

## Extract with uniprot ids

In [24]:
human_id_mapping_uniprot = human_id_mapping.query('source__db_id in @genes_to_ids.uniprot_id')
print(human_id_mapping_uniprot.shape)
human_id_mapping_uniprot.head()

(47, 3)


,source__uniprot_id,source__id_type,source__db_id
73869,P29973,MINT,P29973
137203,P16520,MINT,P16520
143374,P11488,MINT,P11488
260029,P35913,Gene_Name,PDE6B
260088,P35913,BioMuta,PDE6B


In [20]:
"RHO" in human_id_mapping.source__db_id.values

True

## Get ENSMBL ids

In [43]:
visual_uniprot_ids = set(human_id_mapping_uniprot.source__uniprot_id) | set(human_id_mapping_symbols.source__uniprot_id)
len(visual_uniprot_ids)

67

In [44]:
human_id_mapping_visual_system = human_id_mapping.query('source__uniprot_id in @visual_uniprot_ids')
human_id_mapping_visual_system.head()

,source__uniprot_id,source__id_type,source__db_id
73831,P29973,UniProtKB-ID,CNGA1_HUMAN
73832,P29973,Gene_Name,CNGA1
73833,P29973,Gene_Synonym,CNCG
73834,P29973,Gene_Synonym,CNCG1
73835,P29973,GI,2506302


# Concatenate all human mappings

In [27]:
human_retina_ids = pd.concat([human_id_mapping_symbols, human_id_mapping_uniprot], ignore_index=True)
human_retina_ids = human_retina_ids.drop_duplicates()
print(human_retina_ids.shape)
human_retina_ids.head()

(202, 3)


,source__uniprot_id,source__id_type,source__db_id
0,P29973,Gene_Name,CNGA1
1,P29973,BioMuta,CNGA1
2,P29973,GeneCards,CNGA1
3,P29973,GeneReviews,CNGA1
4,P29973,ChiTaRS,CNGA1


## Write human retinal genes with uniprot IDs to disk

In [28]:
pwd

'/mnt/data_sm/olga/kmer-hashing/quest-for-orthologs/data/2019'

In [45]:
human_id_mapping_visual_system.to_csv("human_visual_transduction_with_uniprot_ids.csv", index=False)
human_id_mapping_visual_system.to_csv("human_visual_transduction_with_uniprot_ids.csv.gz", index=False)
human_id_mapping_visual_system.to_parquet("human_visual_transduction_with_uniprot_ids.parquet", index=False)

# Read human proteins and subset to human tfs

In [46]:
retinal_uniprot = set(human_retina_ids.source__uniprot_id)
len(retinal_uniprot)

67

In [47]:
tf_records = []


for filename in glob.iglob('Eukaryota/human-protein-fastas/*.fasta'):
    with screed.open(filename) as records:
        for record in records:
            name = record['name']
            record_id = name.split()[0]
            uniprot_id = record_id.split('|')[1]
            if uniprot_id in retinal_uniprot:
                tf_records.append(record)
print(len(tf_records))


67


In [48]:
tf_records[:3]

[{'name': 'sp|O14610|GBGT2_HUMAN Guanine nucleotide-binding protein G(I)/G(S)/G(O) subunit gamma-T2 OS=Homo sapiens OX=9606 GN=GNGT2 PE=2 SV=1', 'sequence': 'MAQDLSEKDLLKMEVEQLKKEVKNTRIPISKAGKEIKEYVEAQAGNDPFLKGIPEDKNPFKEKGGCLIS', 'description': ''},
 {'name': 'sp|O75916|RGS9_HUMAN Regulator of G-protein signaling 9 OS=Homo sapiens OX=9606 GN=RGS9 PE=1 SV=1', 'sequence': 'MTIRHQGQQYRPRMAFLQKIEALVKDMQNPETGVRMQNQRVLVTSVPHAMTGSDVLQWIVQRLWISSLEAQNLGNFIVRYGYIYPLQDPKNLILKPDGSLYRFQTPYFWPTQQWPAEDTDYAIYLAKRNIKKKGILEEYEKENYNFLNQKMNYKWDFVIMQAKEQYRAGKERNKADRYALDCQEKAYWLVHRCPPGMDNVLDYGLDRVTNPNEVKVNQKQTVVAVKKEIMYYQQALMRSTVKSSVSLGGIVKYSEQFSSNDAIMSGCLPSNPWITDDTQFWDLNAKLVEIPTKMRVERWAFNFSELIRDPKGRQSFQYFLKKEFSGENLGFWEACEDLKYGDQSKVKEKAEEIYKLFLAPGARRWINIDGKTMDITVKGLKHPHRYVLDAAQTHIYMLMKKDSYARYLKSPIYKDMLAKAIEPQETTKKSSTLPFMRRHLRSSPSPVILRQLEEEAKAREAANTVDITQPGQHMAPSPHLTVYTGTCMPPSPSSPFSSSCRSPRKPFASPSRFIRRPSTTICPSPIRVALESSSGLEQKGECSGSMAPRGPSVTESSEASLDTSWPRSRPRAPPKARMALSFSRFLRRGCLASPVFARLSPKCPAVSHGRVQPLGDVGQQLPRLKS

## Write output

In [49]:
human_outdir = 'Eukaryota/human-visual-transduction-fastas/'
! mkdir $human_outdir


with open(f'{human_outdir}/human_visual_transduction_proteins.fasta', 'w') as f:
    for record in tf_records:
        f.write(">{name}\n{sequence}\n".format(**record))

mkdir: cannot create directory ‘Eukaryota/human-visual-transduction-fastas/’: File exists
